In [2]:
import unificar_semanas_sipsa

In [3]:
import pandas as pd
import pickle
import glob

### etapa 0. Unificación de semanas

El objetivo de la función sipsa es unificar y almacenar las información de las bases de datos del SIPSA Y unificarlas para su posterior uso, guardandolas en la carpeta Interim.

In [4]:
unificar_semanas_sipsa.sipsa()

finalizado concatenado de la base 1
Guadando información
finalizado 1 de 3
finalizado concatenado de la base 2
Guadando información
finalizado 2 de 3
finalizado concatenado de la base 3
Guadando información
finalizado 3 de 3


### etapa 1. Analisis de información externa

In [1]:
import pandas as pd
import plotly.express as px
import ipywidgets as widgets
from ipywidgets import interact


In [2]:
df_productos1 = pd.read_excel("../interim/data1.xlsx")

In [3]:
df_productos1.head(2)

,Unnamed: 0.1,Producto,Mercado mayorista,Precio mínimo,Precio máximo,Precio medio,Tendencia,Unnamed: 6,Archivo,Unnamed: 0,Periodo,Fecha
0,0,Acelga,"Bogotá, D.C., Corabastos",400.0,1333.0,719.0,+++,NaN,Semanal-01jul-07jul-2023.xlsx,71,72,2023-07-01
1,1,Acelga,"Bucaramanga, Centroabastos",2000.0,2000.0,2000.0,=,NaN,Semanal-01jul-07jul-2023.xlsx,71,72,2023-07-01


In [7]:
# Función para graficar y filtrar
df_productos1 = df_productos1.sort_values(by='Fecha')
def plot_filtered_graph(tipo_producto,marca):
    filtered_df = df_productos1[(df_productos1['Producto'] == tipo_producto) & (df_productos1['Mercado mayorista'] == marca)]
    fig = px.line(filtered_df, x='Fecha', y='Precio medio', title=f'Precios Promedio - Tipo {tipo_producto}')
    fig.show()

# Widget de selección
tipo_producto_widget = widgets.Dropdown(
    options=df_productos1['Producto'].unique(),
    value=df_productos1['Producto'].unique()[0],
    description='Producto:'
)

tipo_marcador= widgets.Dropdown(
    options=df_productos1['Mercado mayorista'].unique(),
    value=df_productos1['Mercado mayorista'].unique()[0],
    description='Mercado mayorista:'
)

# Interactividad
interact(plot_filtered_graph, tipo_producto=tipo_producto_widget ,marca = tipo_marcador)

interactive(children=(Dropdown(description='Producto:', options=('Fríjol verde cargamanto', 'Acelga', 'Ahuyama…

<function __main__.plot_filtered_graph(tipo_producto, marca)>

### etapa 2. Extraer información de análisis de costos

### etapa 3. Extraer información de análisis de costos

In [13]:
bases = pd.read_csv("../interim/base_consolidada.csv")

In [14]:
bases.head(2)

,Unnamed: 0,nro_orden,fecha,producto,cantidad,precio,descuento,customer_id,sku,product_id_x,...,totalContribucionConDescuento,name,category,region_code,product_category_id,mean_shelf_life,promised_lead_time,purchasing_unit,buy_unit,weight_parameter_apricot
0,0,18120194,2023-01-03,Papa Blanca Sucia Tamaño Mixto Kg,10,2750.0,46750.0,1f1b98af-ee04-4849-a6cf-4fecdf7dba50,BAQ-FRU1-CAT6-234:304:750:770,770,...,-42550.0,Papa Blanca Sucia Tamaño Mixto - KG,Tubérculos,BAQ,1.0,3.0,0.0,48.0,KG,1.0
1,1,18131510,2023-01-03,Tomate Chonto Extramaduro Mixto Kg,1,4998.0,1499.4,8c868951-e6d6-4d63-9592-d4b08ec43f3c,BAQ-FRU1-CAT104105-73423:168376:168377:92108,92108,...,1498.6,Tomate Chonto Extramaduro Mixto - Kg,Verduras,BAQ,1.0,1.0,NaN,22.0,KG,1.0


In [17]:
homologacion = pd.read_excel("../input/homologación_sipsa_frubana_1.xlsx")

In [18]:
homologacion.head(2)

,frubana_name,Sipsa_name,Base_Sipsa
0,Ã‘ame EstÃ¡ndar Kg,Ñame criollo,3.0
1,Ã‘ame Grande Kg,Ñame criollo,3.0


In [36]:
merged_df = pd.merge(bases, homologacion, left_on='producto', right_on='frubana_name', how='inner')
merged_df.head(2)
# Function to calculate the beginning of the week (Sunday)
def beginning_of_week(date):
    return date - pd.Timedelta(days=date.dayofweek)
# Apply the function to the 'Date' column to create the 'Beginning_week' column
merged_df['Beginning_week'] = merged_df['Fecha'].apply(beginning_of_week)
# Display the dataframe
print(merged_df)

KeyError: 'Fecha'

In [24]:
df_productos1.head(2)

,Unnamed: 0.1,Producto,Mercado mayorista,Precio mínimo,Precio máximo,Precio medio,Tendencia,Unnamed: 6,Archivo,Unnamed: 0,Periodo,Fecha
100247,100247,Fríjol verde cargamanto,Chiquinquirá (Boyacá),3000.0,3600.0,3400.0,++,NaN,../input/SIPSA Semanal\Sem_19feb_2022_25feb_20...,0,1,2022-02-19
99785,99785,Acelga,"Bogotá, D.C., Corabastos",400.0,667.0,576.0,+,NaN,../input/SIPSA Semanal\Sem_19feb_2022_25feb_20...,0,1,2022-02-19


In [29]:
df_productos1['Fecha'] = pd.to_datetime(df_productos1['Fecha'])

# Convertir la columna 'fecha' de merged_df a datetime64[ns]
merged_df['fecha'] = pd.to_datetime(merged_df['fecha'])

# Realizar el merge utilizando las columnas convertidas a datetime64[ns]
merged_df2 = pd.merge(df_productos1, merged_df, left_on=['Producto', 'Fecha'], right_on=['Sipsa_name', 'fecha'], how='inner')

# Mostrar las primeras filas del DataFrame resultante
merged_df2.head(2)

,Unnamed: 0.1,Producto,Mercado mayorista,Precio mínimo,Precio máximo,Precio medio,Tendencia,Unnamed: 6,Archivo,Unnamed: 0_x,...,region_code,product_category_id,mean_shelf_life,promised_lead_time,purchasing_unit,buy_unit,weight_parameter_apricot,frubana_name,Sipsa_name,Base_Sipsa
0,96131,Ahuyama,"Cúcuta, Cenabastos",1400.0,1500.0,1490.0,+,NaN,../input/SIPSA Semanal\anex_30jul_al_05ago_202...,23,...,BAQ,1.0,5.0,NaN,20.0,KG,1.0,Ahuyama Grande Kg,Ahuyama,1.0
1,96141,Ahuyama,"Popayán, Plaza de mercado del barrio Bolívar",1740.0,1800.0,1760.0,+,NaN,../input/SIPSA Semanal\anex_30jul_al_05ago_202...,23,...,BAQ,1.0,5.0,NaN,20.0,KG,1.0,Ahuyama Grande Kg,Ahuyama,1.0


In [31]:
merged_df2.columns

Index(['Unnamed: 0.1', 'Producto', 'Mercado mayorista', 'Precio mínimo',
       'Precio máximo', 'Precio medio', 'Tendencia', 'Unnamed: 6', 'Archivo',
       'Unnamed: 0_x', 'Periodo', 'Fecha', 'Unnamed: 0_y', 'nro_orden',
       'fecha', 'producto', 'cantidad', 'precio', 'descuento', 'customer_id',
       'sku', 'product_id_x', 'product_quantity_x_step_unit',
       'product_step_unit', 'product_unit', 'sku_parent', 'month',
       'totalVentasSinDescuento', 'totalVentasConDescuento', 'unit cost',
       'totalContribucionSinDescuento', 'totalContribucionConDescuento',
       'name', 'category', 'region_code', 'product_category_id',
       'mean_shelf_life', 'promised_lead_time', 'purchasing_unit', 'buy_unit',
       'weight_parameter_apricot', 'frubana_name', 'Sipsa_name', 'Base_Sipsa'],
      dtype='object')

In [33]:
Base_total = merged_df2[['Producto','Precio medio','precio','unit cost']].copy()

In [34]:
Base_total.corr()

C:\Users\aabv1\AppData\Local\Temp\ipykernel_7524\3305595094.py:1: FutureWarning:

The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.



,Precio medio,precio,unit cost
Precio medio,1.000000,0.628878,0.622585
precio,0.628878,1.000000,0.997090
unit cost,0.622585,0.997090,1.000000
